## Import Libraries


In [1]:
import os
import glob 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy 
import h5py


from scipy.linalg import hankel
import itertools
import pingouin as pg
import seaborn as sns



%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'


## PC-Algorithm function

In [2]:
def pc_algorithm(X, var_names, lag_zero=True):
    
    #Hyper-parameters
    tau_max = 2
    a = 0.05
    
    #Size of input data
    v = X.shape[1] #number of variables
    n = X.shape[0] #length of time series
    
    #Pre-allocation of Adj-matrix
    linear_index = list(np.arange(0,len(var_names)*(tau_max+1)))
    Adj_matrix = pd.DataFrame(index= linear_index, columns= var_names)
    Adj_matrix[:] = np.inf
    
    #Outer loop to indentify target variable
    for i in range(len(var_names)):
        
        target = X[:,i] #time series of target variable
    
        # List of potnetial parents (All variables are considered, including lag=0)
        parents = np.arange(0,len(var)*(tau_max+1))
    
        # Remove target variable and its lagged versions from the pool of potenital parents
        parents = np.delete(parents, np.arange(i, len(parents), len(var)))
        
        #Iterate over potential parents
        for j in range(len(parents)):
            
            idx = parents[j] #linear index of parent
        
            #time series of the source
            #variable is given by the remainder idx%v
            #lag is given by the quotient idx//v
            source = hankel(X[:,idx%v])[:,idx//v] 
        
            # Construct the pool of conditioning variables Z
            # where Z = parents - source variable - lagged versions of source variable
            Z = parents + 1 - 1
            removed = np.arange(parents[j], len(var)*(tau_max+1), len(var))
            Z[np.isin(Z, removed)] = -99
            Z = Z[Z > -99]
        
            exit = 0
            
            #Inner loop increase the size of conditioning set iteratively from 0 to len(Z)
            for Nz in range(len(Z)):
                
                # All possible combinations of size Nz that can be conditioned on
                combinations = list(itertools.combinations(Z, Nz))
                
                for k in range(len(combinations)):
                    
                    cond_set = np.ones([n, Nz])
                    
                    for ii in range(Nz):
                        
                        idx = combinations[k][ii]
                        cond_set[:,ii] = hankel(X[:,idx%v])[:,idx//v]
                        
                    ### Run the conditional Indepence test: I(target, sources, cond_set) ###
                
                    print(str(i) + ', ' + str(parents[j]) + '; ' + '[' + str(combinations[k]) + ']')
                    
                    # Concatenate arrays by column in the order: target, sources, cond_set
                    data = np.concatenate((target.reshape(n,1), source.reshape(n,1), cond_set), axis=1)
                    data = pd.DataFrame(data, columns= list(range(data.shape[1])))
                    
                    # Partial correlation
                    result = pg.partial_corr(data=data, x=0, y=1, covar=list(np.arange(2,data.shape[1]))
                                             , method= 'spearman').round(3)
                    
                    corr = result['r'][0]
                    p_value = result['p-val'][0]
                    
                    if (p_value > a):
                        Adj_matrix.iloc[parents[j],i] = np.nan #no causal link exists
                        exit = 1
                        break
                    elif (p_value <= a):
                        I_min = min(Adj_matrix.iloc[parents[j],i], corr)
                        Adj_matrix.iloc[parents[j],i] = I_min
                        
                if exit==1:
                    break
                    
        
    return Adj_matrix
        